In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier



In [2]:
train = pd.read_csv('Data Mining VU data/training_set_VU_DM_2014.csv')

In [3]:
test = pd.read_csv('Data Mining VU data/test_set_VU_DM_2014.csv')

In [4]:
# smaller samples for faster computing
train_sample = train.sample(frac=0.5, random_state=0)
test_sample = train.drop(train_sample.index)

In [5]:
train_sample.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
930452,62094,2013-01-26 22:10:19,18,31,NaN,NaN,32,47040,4,4.5,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,0
1701430,114235,2012-11-13 12:42:21,1,13,NaN,NaN,15,115700,5,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0
1595996,107254,2013-04-25 10:22:32,16,31,NaN,NaN,31,31574,4,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0
3367696,226202,2013-02-21 19:02:17,5,219,NaN,NaN,219,33423,3,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0
3880444,260461,2013-05-02 23:11:06,14,100,NaN,NaN,219,24194,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0


In [6]:
target = pd.DataFrame(train_sample['booking_bool'])
data = pd.DataFrame(train_sample[['site_id', 'prop_country_id']])


clf = RandomForestClassifier()
clf.fit(data, target)


/Users/Tamika/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [10]:
from sklearn.metrics import accuracy_score

train_sample_2 = train.sample(frac=0.7, random_state=0)

data_test = pd.DataFrame(train_sample_2[['site_id', 'prop_country_id']])
target_test = pd.DataFrame(train_sample_2['booking_bool'])

predicted = clf.predict(data_test)
accuracy = accuracy_score(target_test, predicted)

print('accuracy: %f' %accuracy)



accuracy: 0.972170


In [11]:
train_clean = pd.read_csv('processed_data/train_clean.csv')
test_clean = pd.read_csv('processed_data/test_clean.csv')

In [12]:
train_clean.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,click_bool,gross_bookings_usd,booking_bool,star_diff,star_diff_bool,price_diff,price_diff_bool,visitor_hist_starrating_filled,visitor_hist_adr_usd_filled,willingness_to_pay
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,0,NaN,0,NaN,0,NaN,0,3.472818,202.072673,0
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,0,NaN,0,NaN,0,NaN,0,3.472818,202.072673,0
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,0,NaN,0,NaN,0,NaN,0,3.472818,202.072673,0
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,...,0,NaN,0,NaN,0,NaN,0,3.472818,202.072673,0
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,0,NaN,0,NaN,0,NaN,0,3.472818,202.072673,0


In [18]:
columns = [col for col in train_clean.columns if col not in ['booking_bool', 'date_time']]
col = list(train_clean.columns.values)

print(col)
print(columns)

['srch_id', 'date_time', 'site_id', 'visitor_location_country_id', 'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id', 'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool', 'prop_location_score1', 'prop_location_score2', 'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag', 'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count', 'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool', 'srch_query_affinity_score', 'orig_destination_distance', 'random_bool', 'click_bool', 'gross_bookings_usd', 'booking_bool', 'star_diff', 'star_diff_bool', 'price_diff', 'price_diff_bool', 'visitor_hist_starrating_filled', 'visitor_hist_adr_usd_filled', 'willingness_to_pay']
['srch_id', 'site_id', 'visitor_location_country_id', 'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id', 'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool', 'prop_location_score1', 'prop_location_scor

In [21]:
train_sample3 = train_clean.sample(frac=0.5, random_state=0)
test_sample3 = train_clean.drop(train_sample3.index)

In [23]:
#pd.isnull(train_sample3.sum()
train_sample3.isnull().sum()

srch_id                                 0
date_time                               0
site_id                                 0
visitor_location_country_id             0
visitor_hist_starrating           2353405
visitor_hist_adr_usd              2352835
prop_country_id                         0
prop_id                                 0
prop_starrating                         0
prop_review_score                       0
prop_brand_bool                         0
prop_location_score1                    0
prop_location_score2                    0
prop_log_historical_price               0
position                                0
price_usd                               0
promotion_flag                          0
srch_destination_id                     0
srch_length_of_stay                     0
srch_booking_window                     0
srch_adults_count                       0
srch_children_count                     0
srch_room_count                         0
srch_saturday_night_bool          

In [26]:
columns = [col for col in train_sample3.columns if col not in ['booking_bool', 'date_time', 'visitor_hist_starrating', 'visitor_hist_adr_usd', 'gross_bookings_usd', 'star_diff', 'price_diff']]

target = pd.DataFrame(train_sample3['booking_bool'])
data = pd.DataFrame(train_sample3[columns])

clf = RandomForestClassifier()
clf.fit(data, target)

target_test = pd.DataFrame(train_clean['booking_bool'])
predicted = clf.predict(train_clean[columns])
accuracy = accuracy_score(target_test, predicted)

print('accuracy: %f' %accuracy)

/Users/Tamika/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


accuracy: 0.996301


In [28]:
from sklearn.metrics import f1_score

f1_ = f1_score(target_test, predicted)
print(f1_)


0.9344497812848442


In [31]:
print(columns)
clf.feature_importances_

dictionary = dict(zip(columns, clf.feature_importances_))

['srch_id', 'site_id', 'visitor_location_country_id', 'prop_country_id', 'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool', 'prop_location_score1', 'prop_location_score2', 'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag', 'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count', 'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool', 'srch_query_affinity_score', 'orig_destination_distance', 'random_bool', 'click_bool', 'star_diff_bool', 'price_diff_bool', 'visitor_hist_starrating_filled', 'visitor_hist_adr_usd_filled', 'willingness_to_pay']


In [39]:
#pd.DataFrame.from_dict(dictionary)
pd.Series(dictionary).to_frame()

,0
click_bool,0.671961
orig_destination_distance,0.015095
position,0.028983
price_diff_bool,0.000295
price_usd,0.018353
promotion_flag,0.002104
prop_brand_bool,0.002180
prop_country_id,0.006690
prop_id,0.016973
prop_location_score1,0.014957
